In [59]:
#@title Manual Data Injection
from six.moves import cPickle as pickle
import platform
import numpy as np
 
def load_files(filenames):
    data = np.array([])
    labels = np.array([])
    for name in filenames:
        with open(name, 'rb') as f:
            mydict = pickle.load(f, encoding='latin1')

        # The labels have different names in the two datasets.
        label_func = lambda x: np.array(x['fine_labels'], dtype='int32')
        newlabels = label_func(mydict)
        if data.size:
            data = np.vstack([data, mydict['data']])
            labels = np.hstack([labels, newlabels])
        else:
            data = mydict['data']
            labels = newlabels
    data = np.reshape(data, [-1, 3, 32, 32], order='C')
    data = np.transpose(data, [0, 2, 3, 1])

    return data, labels

def load_CIFAR100(data_dir):
    """ load all of cifar """
    train_files = ['train']
    train_files = [os.path.join(data_dir, f) for f in train_files]
    test_files = ['test']
    test_files = [os.path.join(data_dir, f) for f in test_files]
    num_classes = 100
    train_data, train_labels = load_files(train_files)
    test_data, test_labels = load_files(test_files)

    return train_data, train_labels, test_data, test_labels
 
def get_CIFAR100_data(val_size=2000):
    # Load the raw CIFAR-100 data
    cifar100_dir = r'C:\Users\Pegah Khazaie\Desktop\cifar-100-python'
    x_train, y_train, x_test, y_test = load_CIFAR100(cifar100_dir)
    # Subsample the data
 
    x_train, x_val = np.split(x_train,
                              [x_train.shape[0]-val_size])
    y_train, y_val = np.split(y_train,
                              [y_train.shape[0]-val_size])
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')
 
#     x_train /= 255
#     x_val /= 255
#     x_test /= 255

    return x_train, y_train, x_val, y_val, x_test, y_test
 
# Invoke the above function to get our data.
x_train, y_train, x_val, y_val, x_test, y_test = get_CIFAR100_data()
 
print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (48000, 32, 32, 3)
Train labels shape:  (48000,)
Validation data shape:  (2000, 32, 32, 3)
Validation labels shape:  (2000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [60]:
from keras.datasets import cifar100
from keras.models import *
from keras.layers import *
import numpy as np
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.preprocessing import image
from keras import regularizers,optimizers
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch < 2:
        lrate = 0.005
    if epoch > 5:
        lrate = 0.0001
    return lrate
 

In [61]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


In [64]:
model  = Sequential()
model.add(Conv2D(32,(2,2),padding='same',input_shape=(32,32,3),activation='elu'))
model.add(MaxPool2D(2,2))    
model.add(Conv2D(64, (2, 2), padding='same',activation='elu'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100,activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.rmsprop(lr=0.001, decay=0.00005), metrics=['accuracy'])

In [65]:
callbacks = [EarlyStopping(monitor='val_loss', patience=1, mode='min', verbose=1)]

In [66]:
model.fit(x_train, y_train,epochs=10,validation_split=0.2,verbose=1,callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 38400 samples, validate on 9600 samples
Epoch 1/10
38400/38400 [==============================] - 135s 4ms/step - loss: 0.0553 - accuracy: 0.9900 - val_loss: 0.0530 - val_accuracy: 0.9900
Epoch 2/10
38400/38400 [==============================] - 135s 4ms/step - loss: 0.0539 - accuracy: 0.9900 - val_loss: 0.0522 - val_accuracy: 0.9900
Epoch 3/10
38400/38400 [==============================] - 129s 3ms/step - loss: 0.0534 - accuracy: 0.9900 - val_loss: 0.0512 - val_accuracy: 0.9900
Epoch 4/10
38400/38400 [==============================] - 131s 3ms/step - loss: 0.0526 - accuracy: 0.9900 - val_loss: 0.0503 - val_accuracy: 0.9900
Epoch 5/10
38400/38400 [==============================] - 133s 3ms/step - loss: 0.0520 - accuracy: 0.9900 - val_loss: 0.0510 - val_accuracy: 0.9900
Epoch 00005: early stopping


In [67]:
res_acc = model.evaluate(x_test,y_test)

10000/10000 [==============================] - 9s 923us/step


In [68]:
print("res_acc:",res_acc[1])

res_acc: 0.9899795055389404
